In [2]:
pip install snowflake-snowpark-python

  Obtaining dependency information for snowflake-snowpark-python from https://files.pythonhosted.org/packages/1c/1a/35a3b9126d1c74f10ab50db6218ebd92aeb6f9d73f4e0b7b77241e853277/snowflake_snowpark_python-1.13.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/53.8 kB ? eta -:--:--
     --------------------- ---------------- 30.7/53.8 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 53.8/53.8 kB 926.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/404.6 kB ? eta -:--:--
   --------------------------------------  399.4/404.6 kB 12.6 MB/s eta 0:00:01
   --------------------------------------- 404.6/404.6 kB 12.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col,avg,row_number
from snowflake.snowpark import Window

def initiateSession(): 
    
    connection_parameters = {
            "account": "rn65832.ap-southeast-1",
            "user": "  18",
            "password": "  @0409",
            "role": "Accountadmin", 
            "warehouse": "compute_wh"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

In [4]:
df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS")

In [5]:
type(df_orders)

snowflake.snowpark.table.Table

In [6]:
df_orders.show(5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"O_ORDERKEY"  |"O_CUSTKEY"  |"O_ORDERSTATUS"  |"O_TOTALPRICE"  |"O_ORDERDATE"  |"O_ORDERPRIORITY"  |"O_CLERK"        |"O_SHIPPRIORITY"  |"O_COMMENT"                                         |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|3393217       |702002       |F                |61041.85        |1995-01-11     |2-HIGH             |Clerk#000000520  |0                 |accounts wake furiousl                              |
|3393218       |1041638      |F                |37762.70        |1994-04-18     |1-URGENT           |Clerk#000004446  |0                 |e slyly carefully pending accounts. silent, reg...  |
|3393219       |298181       |F         

In [7]:
df_orders.groupBy("O_ORDERSTATUS").avg("O_TOTALPRICE").show()

-----------------------------------------
|"O_ORDERSTATUS"  |"AVG(O_TOTALPRICE)"  |
-----------------------------------------
|P                |184843.54565642      |
|O                |150183.65237497      |
|F                |150218.74052087      |
-----------------------------------------



In [8]:
df_orders.groupBy("O_ORDERSTATUS").count().show()

-----------------------------
|"O_ORDERSTATUS"  |"COUNT"  |
-----------------------------
|P                |383352   |
|F                |7309184  |
|O                |7307464  |
-----------------------------



In [9]:
df_tmp = session.create_dataframe([("iphone12", "2000","phone","electronic gadgets"), \
                                   ("iphone13", "2400","phone","electronic gadgets"), \
                                   ("sansung galaxy", "2600","phone","electronic gadgets"), \
                                   ("microsoft windows 11", "3600","phone","electronic gadgets"), \
                                   ("samsung galaxy watch", "2600","watch","electronic gadgets"), \
                                   ("samsung galaxy watch2", "2900","watch","electronic gadgets"), \
                                   ("apple watch series 8", "3200","watch","electronic gadgets"), \
                                   ("apple watch se2", "1800","watch","electronic gadgets"), \
                                    ("Nike White Jersey", "300","clothing","non-electronic"), \
                                   ("Air jordon Shoes", "2300","clothing","non-electronic"), \
                                   ("Reebok Sneakers", "3000","clothing","non-electronic"), \
                                   ("yeezy red sneakers", "5300","clothing","non-electronic") \
                                  ], \
                                  schema=["product_name", "value","category","product_type"])

df_tmp.show()

---------------------------------------------------------------------
|"PRODUCT_NAME"         |"VALUE"  |"CATEGORY"  |"PRODUCT_TYPE"      |
---------------------------------------------------------------------
|iphone12               |2000     |phone       |electronic gadgets  |
|iphone13               |2400     |phone       |electronic gadgets  |
|sansung galaxy         |2600     |phone       |electronic gadgets  |
|microsoft windows 11   |3600     |phone       |electronic gadgets  |
|samsung galaxy watch   |2600     |watch       |electronic gadgets  |
|samsung galaxy watch2  |2900     |watch       |electronic gadgets  |
|apple watch series 8   |3200     |watch       |electronic gadgets  |
|apple watch se2        |1800     |watch       |electronic gadgets  |
|Nike White Jersey      |300      |clothing    |non-electronic      |
|Air jordon Shoes       |2300     |clothing    |non-electronic      |
---------------------------------------------------------------------



In [10]:
window = Window.order_by("value") # Without partition by 

window2 = Window.partition_by("product_type").order_by("value") # Using partition by clause

window3 = Window.partition_by(['category','product_type']).order_by("value") # Using multiple columns in partition-by clause


In [11]:
df_tmp.withColumn("row_number",row_number().over(window)).show()

------------------------------------------------------------------------------------
|"PRODUCT_NAME"         |"VALUE"  |"CATEGORY"  |"PRODUCT_TYPE"      |"ROW_NUMBER"  |
------------------------------------------------------------------------------------
|apple watch se2        |1800     |watch       |electronic gadgets  |1             |
|iphone12               |2000     |phone       |electronic gadgets  |2             |
|Air jordon Shoes       |2300     |clothing    |non-electronic      |3             |
|iphone13               |2400     |phone       |electronic gadgets  |4             |
|sansung galaxy         |2600     |phone       |electronic gadgets  |5             |
|samsung galaxy watch   |2600     |watch       |electronic gadgets  |6             |
|samsung galaxy watch2  |2900     |watch       |electronic gadgets  |7             |
|Nike White Jersey      |300      |clothing    |non-electronic      |8             |
|Reebok Sneakers        |3000     |clothing    |non-electronic   

In [12]:
df_tmp.withColumn("row_number",row_number().over(window2)).show()

------------------------------------------------------------------------------------
|"PRODUCT_NAME"         |"VALUE"  |"CATEGORY"  |"PRODUCT_TYPE"      |"ROW_NUMBER"  |
------------------------------------------------------------------------------------
|apple watch se2        |1800     |watch       |electronic gadgets  |1             |
|iphone12               |2000     |phone       |electronic gadgets  |2             |
|iphone13               |2400     |phone       |electronic gadgets  |3             |
|sansung galaxy         |2600     |phone       |electronic gadgets  |4             |
|samsung galaxy watch   |2600     |watch       |electronic gadgets  |5             |
|samsung galaxy watch2  |2900     |watch       |electronic gadgets  |6             |
|apple watch series 8   |3200     |watch       |electronic gadgets  |7             |
|microsoft windows 11   |3600     |phone       |electronic gadgets  |8             |
|Air jordon Shoes       |2300     |clothing    |non-electronic   

In [13]:
df_tmp.withColumn("row_number",row_number().over(window3)).show()

------------------------------------------------------------------------------------
|"PRODUCT_NAME"         |"VALUE"  |"CATEGORY"  |"PRODUCT_TYPE"      |"ROW_NUMBER"  |
------------------------------------------------------------------------------------
|iphone12               |2000     |phone       |electronic gadgets  |1             |
|iphone13               |2400     |phone       |electronic gadgets  |2             |
|sansung galaxy         |2600     |phone       |electronic gadgets  |3             |
|microsoft windows 11   |3600     |phone       |electronic gadgets  |4             |
|apple watch se2        |1800     |watch       |electronic gadgets  |1             |
|samsung galaxy watch   |2600     |watch       |electronic gadgets  |2             |
|samsung galaxy watch2  |2900     |watch       |electronic gadgets  |3             |
|apple watch series 8   |3200     |watch       |electronic gadgets  |4             |
|Air jordon Shoes       |2300     |clothing    |non-electronic   

In [14]:
df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS") \
            .select(['O_ORDERKEY','O_CUSTKEY','O_TOTALPRICE']) \
            .filter("O_ORDERKEY=31593217")

df_orders.show()

-----------------------------------------------
|"O_ORDERKEY"  |"O_CUSTKEY"  |"O_TOTALPRICE"  |
-----------------------------------------------
|31593217      |1095217      |119544.06       |
-----------------------------------------------



In [15]:
sql = """
        select 
            S_SUPPKEY as SUPP_KEY,
            S_NAME as SUPPLIER_NAME,
            S_NATIONKEY as NATION_KEY,
            N.N_NAME as NATION_NAME
        from 
            SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.SUPPLIER S 
        LEFT JOIN 
            SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.NATION N
        ON 
            S.S_NATIONKEY = N.N_NATIONKEY
        """

df_supplier = session.sql(sql)
df_supplier.show()

------------------------------------------------------------------
|"SUPP_KEY"  |"SUPPLIER_NAME"     |"NATION_KEY"  |"NATION_NAME"  |
------------------------------------------------------------------
|15001       |Supplier#000015001  |19            |ROMANIA        |
|15002       |Supplier#000015002  |7             |GERMANY        |
|15003       |Supplier#000015003  |7             |GERMANY        |
|15004       |Supplier#000015004  |9             |INDONESIA      |
|15005       |Supplier#000015005  |6             |FRANCE         |
|15006       |Supplier#000015006  |16            |MOZAMBIQUE     |
|15007       |Supplier#000015007  |10            |IRAN           |
|15008       |Supplier#000015008  |9             |INDONESIA      |
|15009       |Supplier#000015009  |9             |INDONESIA      |
|15010       |Supplier#000015010  |1             |ARGENTINA      |
------------------------------------------------------------------



In [16]:
type(df_supplier)

snowflake.snowpark.dataframe.DataFrame

In [17]:
import pandas as pd 
df_supplier = df_supplier.to_pandas()
df_supplier.columns = [x.upper() for x in df_supplier.columns]

In [18]:
type(df_supplier)

pandas.core.frame.DataFrame

In [20]:
query = """create or replace table SNOWPARK_  .test_schema.SUPPLIER_DETAILS (supp_key integer,supplier_name varchar(100),
            nation_key integer ,nation_name varchar(100) )
        """
session.sql(query).collect()


[Row(status='Table SUPPLIER_DETAILS successfully created.')]

In [21]:
session.sql("use schema SNOWPARK_  .test_schema").collect()
res = session.write_pandas(df_supplier, "SUPPLIER_DETAILS")

In [22]:
session.close()